In [83]:
import pandas as pd
 
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
 
# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
 
# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다.
code_df = code_df[['회사명', '종목코드']]
 
# 한글로된 컬럼명을 영어로 바꿔준다.
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})
# code_df.head()
tmp = code_df.query('name=="대한한공"')
tmp['code'].to_string(index=False)

'Series([], )'

In [69]:
# 종목 이름을 입력하면 종목에 해당하는 코드를 불러와
# 네이버 금융(http://finance.naver.com)에 넣어줌
def get_url(item_name, code_df):

    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False).strip()
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
    
    print("요청 URL = {}".format(url))
    return url
    
 
# 신라젠의 일자데이터 url 가져오기
item_name='대한항공'
url = get_url(item_name, code_df)
 
# 일자 데이터를 담을 df라는 DataFrame 정의
df = pd.DataFrame()
 
# 1페이지에서 20페이지의 데이터만 가져오기
for page in range(1, 11):
    pg_url = '{url}&page={page}'.format(url=url, page=page)
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)
 
# df.dropna()를 이용해 결측값 있는 행 제거
df = df.dropna()
 
# 상위 5개 데이터 확인하기
df.head()



요청 URL = http://finance.naver.com/item/sise_day.nhn?code=003490


,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.02.24,22300.0,1250.0,22800.0,22850.0,22200.0,837966.0
2,2020.02.21,23550.0,100.0,23500.0,23900.0,23250.0,727771.0
3,2020.02.20,23650.0,550.0,24450.0,24700.0,23650.0,959631.0
4,2020.02.19,24200.0,250.0,24850.0,24850.0,24000.0,681276.0
5,2020.02.18,24450.0,850.0,25250.0,25250.0,24400.0,891527.0


In [70]:
# 한글로 된 컬럼명을 영어로 바꿔줌
df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', 
                         '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})
 
# 데이터의 타입을 int형으로 바꿔줌
df[['close', 'diff', 'open', 'high', 'low', 'volume']] \
    = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)
 
# 컬럼명 'date'의 타입을 date로 바꿔줌
df['date'] = pd.to_datetime(df['date'])
 
# 일자(date)를 기준으로 오름차순 정렬
df = df.sort_values(by=['date'], ascending=True)
 
# 상위 5개 데이터 확인
df.head()


,date,close,diff,open,high,low,volume
148,2019-09-27,23050,150,23000,23200,22650,322423
147,2019-09-30,22900,150,22850,23150,22800,260724
146,2019-10-01,23450,550,22800,23550,22750,365477
145,2019-10-02,22950,500,23300,23450,22900,348555
144,2019-10-04,22800,150,22900,23100,22700,267479


In [71]:
# 일자(n,m,t)에 따른 Stochastic(KDJ)의 값을 구하기 위해 함수형태로 만듬 
def get_stochastic(df, n=15, m=5, t=3):
    
    # 입력받은 값이 dataframe이라는 것을 정의해줌
    df = pd.DataFrame(df)
    
    # n일중 최고가
    ndays_high = df.high.rolling(window=n, min_periods=1).max()
    # n일중 최저가
    ndays_low = df.low.rolling(window=n, min_periods=1).min()
 
    # Fast%K 계산
    kdj_k = ((df.close - ndays_low) / (ndays_high - ndays_low))*100
    # Fast%D (=Slow%K) 계산
    kdj_d = kdj_k.ewm(span=m).mean()
    # Slow%D 계산
    kdj_j = kdj_d.ewm(span=t).mean()
 
    # dataframe에 컬럼 추가
    df = df.assign(kdj_k=kdj_k, kdj_d=kdj_d, kdj_j=kdj_j).dropna()
    
    return df


In [72]:
df = get_stochastic(df)
df.head()



,date,close,diff,open,high,low,volume,kdj_k,kdj_d,kdj_j
148,2019-09-27,23050,150,23000,23200,22650,322423,72.727273,72.727273,72.727273
147,2019-09-30,22900,150,22850,23150,22800,260724,45.454545,56.363636,61.818182
146,2019-10-01,23450,550,22800,23550,22750,365477,88.888889,71.770335,67.505126
145,2019-10-02,22950,500,23300,23450,22900,348555,33.333333,55.804196,61.264630
144,2019-10-04,22800,150,22900,23100,22700,267479,16.666667,40.779836,50.691833


In [73]:
df.tail(1)

,date,close,diff,open,high,low,volume,kdj_k,kdj_d,kdj_j
1,2020-02-24,22300,1250,22800,22850,22200,837966,1.785714,15.896928,23.376654


In [74]:
import plotly.offline as offline 
import plotly.graph_objs as go
from plotly import subplots
 
# jupyter notebook 에서 출력 
offline.init_notebook_mode(connected=True)
 
kdj_k = go.Scatter(
                x=df.date,
                y=df['kdj_k'],
                name="Fast%K")
 
kdj_d = go.Scatter(
                x=df.date,
                y=df['kdj_d'],
                name="Fast%D")
 
kdj_d2 = go.Scatter(
                x=df.date,
                y=df['kdj_d'],
                name="Slow%K")
 
kdj_j = go.Scatter(
                x=df.date,
                y=df['kdj_j'],
                name="Slow%D")
 
 
trade_volume = go.Bar(
                x=df.date,
                y=df['volume'],
                name="volume")
 
 
# data = [kdj_k, kdj_d]
data1 = [kdj_d2, kdj_j]
data2 = [trade_volume]
 
# data = [celltrion]
# layout = go.Layout(yaxis=dict(
#         autotick=False,
#         ticks='outside',
#         tick0=0,
#         dtick=10,
#         ticklen=8,
#         tickwidth=4,
#         tickcolor='#000'
#     ))
 
fig = subplots.make_subplots(rows=2, cols=1, shared_xaxes=True)
 
for trace in data1:
    fig.append_trace(trace, 1,1)
 
for trace in data2:
    fig.append_trace(trace, 2,1)
# fig = go.Figure(data=data, layout=layout)
 
offline.iplot(fig)

